In [1]:
pip install azureml-pipeline

Note: you may need to restart the kernel to use updated packages.


In [2]:
from azureml.core import Workspace, Datastore, Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

# Initialize workspace and compute

In [22]:
# Initialize workspace
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json")

# Define compute target
compute_name = "Onlineretail2"
compute_target = ws.compute_targets[compute_name]

# Define Environment

In [23]:
env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")
env.register(workspace=ws)

{
    "assetId": "azureml://locations/eastus2/workspaces/7ce76f32-f5fe-4dd3-b900-96fb92b58ebb/environments/myenv/versions/13",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
      

In [24]:
env = Environment.get(workspace=ws, name="myenv")
print(env.python.conda_dependencies.serialize_to_string())


name: myenv
channels:
- defaults
dependencies:
- python=3.7
- scikit-learn
- statsmodels
- pip
- pip:
  - pmdarima
  - azureml-sdk
  - azureml-dataset-runtime[fuse, pandas]
  - azureml-defaults
  - joblib
  - scikit-learn
  - matplotlib



# Define run configuration


In [25]:
aml_config = RunConfiguration()
aml_config.target = compute_target
aml_config.environment = env

# Pipeline


In [28]:
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

source_directory = ""

#Script initialization
py_script_run_read = PythonScriptStep(
                name='Data Wrangling Step',
                script_name = read_data,
                compute_target = compute_target,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps = pipeline_steps)

# Experiment 1


In [29]:
experiment_name = "Final_expt2"
experiment = Experiment(ws, experiment_name)
pipeline_run = experiment.submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Created step Data Wrangling Step [ab95bf75][fe388afe-642d-4276-bb6f-b3e23c3bbcfe], (This step will run and generate new outputs)
Created step preprocessing.py [817b2bbd][500d8ebc-a695-4286-8803-926485a6becc], (This step will run and generate new outputs)
Created step modelling.py [3d95123f][95671390-46ca-4e38-89e8-8c8156118f4e], (This step will run and generate new outputs)
Submitted PipelineRun 917e38df-7d3b-4f0c-ac1e-b836d5ed51e8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/917e38df-7d3b-4f0c-ac1e-b836d5ed51e8?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 917e38df-7d3b-4f0c-ac1e-b836d5ed51e8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/917e38df-7d3b-4f0c-ac1e-b836d5ed51e8?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
Pipeline

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_9056511165ce76325fbbfe9cbc33e288/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error:   File \"/azureml-envs/azureml_9056511165ce76325fbbfe9cbc33e288/lib/python3.7/site-packages/statsmodels/regression/linear_model.py\", line 907, in __init__\n    hasconst=hasconst, **kwargs)\n  File \"/azureml-envs/azureml_9056511165ce76325fbbfe9cbc33e288/lib/python3.7/site-packages/statsmodels/regression/linear_model.py\", line 734, in __init__\n    weights=weights, hasconst=hasconst, **kwargs)\n  File \"/azureml-envs/azureml_9056511165ce76325fbbfe9cbc33e288/lib/python3.7/site-packages/statsmodels/regression/linear_model.py\", line 190, in __init__\n    super(RegressionModel, self).__init__(endog, exog, **kwargs)\n  File \"/azureml-envs/azureml_905651

In [ ]:
pipeline_run

# Download the model file from the models folder


In [14]:
# Retrieve the datastore
datastore = Datastore.get(ws, 'workspaceblobstore')

# Download the model file from the models folder
datastore.download(target_path="model", 
                   prefix="models/best_autoarima_model.pkl",
                   overwrite=True,  # Set to True to overwrite existing files
                   show_progress=True)


Downloaded models/best_autoarima_model.pkl, 1 files out of an estimated total of 1


1

# Registering the model


In [15]:
from azureml.core import Datastore, Workspace, Model

model = Model.register(workspace=ws,
                       model_name='best_autoarima_model',  # Name of the model in the Azure registry
                       model_path="model/models/best_autoarima_model.pkl",  # Path to the downloaded model file
                       description="Best Auto-ARIMA model for sales forecasting",
                       tags={"model_type": "Auto-ARIMA", "framework": "pmdarima"})

print(f"Model registered: {model.name} with version {model.version}")


Registering model best_autoarima_model
Model registered: best_autoarima_model with version 4


# Load the model


In [16]:
from azureml.core.model import Model
model = Model(ws, name='best_autoarima_model')
print("Loaded model version:", model.version)

Loaded model version: 4


# Deployment


In [70]:
%%writefile score.py
import joblib
import json
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model_3
    model_3_path = Model.get_model_path(model_name='best_autoarima_model')
    model_3 = joblib.load(model_3_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        
        data_df = pd.DataFrame(data, columns=['InvoiceDate', 'Sales'])
        
        sales_data = data_df['Sales'].values
        
        # Define how many future periods you want to predict
        n_periods = 10 
        
        result_1 = model_3.predict(n_periods=n_periods)
        
        return {"predictions": result_1.tolist()}
    except Exception as e:
        result = str(e)
        return {"error": result}




Overwriting score.py


# Inferencing the Model

In [71]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

In [72]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-modelforecasting"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

/tmp/ipykernel_3474/3772987190.py:7: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-08-18 16:49:44+00:00 Creating Container Registry if not exists.
2024-08-18 16:49:48+00:00 Use the existing image.
2024-08-18 16:49:50+00:00 Submitting deployment to compute..
2024-08-18 16:49:53+00:00 Checking the status of deployment aciservice-modelforecasting..
2024-08-18 16:52:48+00:00 Checking the status of inference endpoint aciservice-modelforecasting.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
